<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%8F%89%EA%B7%A0_%EA%B8%B0%EC%98%A8_%EC%98%88%EC%B8%A1_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

data = pd.read_csv('/content/train.csv')
submission = pd.read_csv('/content/sample_submission.csv')

In [4]:
from datetime import datetime

# '일시' 열을 날짜 데이터로 변환
data['일시'] = pd.to_datetime(data['일시'])

data['연도'] = data['일시'].dt.year
data['월'] = data['일시'].dt.month
data['일'] = data['일시'].dt.day

# 연, 월 별 중앙값
monthly_medians = data.groupby('월').median()
yearly_medians = data.groupby('연도').median()
# 연, 월 별 평균값
monthly_means = data.groupby('월').mean()
yearly_means = data.groupby('연도').mean()

<ipython-input-4-e2deca90dd51>:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  monthly_medians = data.groupby('월').median()
<ipython-input-4-e2deca90dd51>:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  yearly_medians = data.groupby('연도').median()
<ipython-input-4-e2deca90dd51>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  monthly_means = data.groupby('월').mean()
<ipython-input-4-e2deca90dd51>:15: FutureWa

In [6]:
current_year = datetime.now().year

# 2월 29일에 해당하는 행 선택
selected_rows = data[(data['일시'].dt.month == 2) & (data['일시'].dt.day == 29)]

# 29일에 해당하는 16개 행
# 23011 - 16 = 22995

# 선택한 행의 인덱스를 사용하여 삭제
data = data.drop(selected_rows.index)

In [7]:
# 결측값 대체
data.fillna(monthly_medians, inplace=True)

for column in data.columns:
    if data[column].isnull().any():  # 결측값이 있는 경우에만 처리
        data[column+'_중앙값'] = data.apply(lambda row: monthly_medians.loc[row['월'], column], axis=1)
        data[column].fillna(data[column+'_중앙값'], inplace=True)
        data.drop(column+'_중앙값', axis=1, inplace=True)

In [8]:
train = data[['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율']]

target = data['평균기온']

import warnings
warnings.filterwarnings(action='ignore')

In [9]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train)

train_scaled = ss.transform(train)

In [10]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(train_scaled, target)

RandomForestRegressor()

In [11]:
pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 556.7 kB/s eta 0:00:00


In [12]:
from skopt import BayesSearchCV

# 탐색할 하이퍼파라미터 값 지정 (None 값을 다른 값으로 대체)
param_space = {
    'n_estimators': (50, 200),
    'max_depth': (1, 30),  # None을 적절한 값으로 대체
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 4),
}

# Bayesian Optimization을 사용하여 하이퍼파라미터 탐색
opt = BayesSearchCV(rf, param_space, n_iter=10, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', random_state=42)
opt.fit(train_scaled, target)

# 최적의 하이퍼파라미터 출력
print("Best Parameters: ", opt.best_params_)

Best Parameters:  OrderedDict([('max_depth', 25), ('min_samples_leaf', 4), ('min_samples_split', 4), ('n_estimators', 193)])


In [13]:
rf = RandomForestRegressor(max_depth = 25, min_samples_leaf = 4, min_samples_split = 4, n_estimators = 193)

rf.fit(train_scaled, target)

RandomForestRegressor(max_depth=25, min_samples_leaf=4, min_samples_split=4,
                      n_estimators=193)

In [14]:
rf_pred = rf.predict(train_scaled)

from sklearn.metrics import mean_absolute_error

rf_mae = mean_absolute_error(rf_pred, target)

print(rf_mae)

0.23152088610416707


In [15]:
submission['평균기온'] = rf_pred[:358]
print(submission.head(30))

            일시       평균기온
0   2023-01-01  -1.695697
1   2023-01-02  -2.177952
2   2023-01-03   3.460828
3   2023-01-04   6.425697
4   2023-01-05  -3.931063
5   2023-01-06  -5.321892
6   2023-01-07   1.148911
7   2023-01-08   1.845776
8   2023-01-09   2.297040
9   2023-01-10   1.928083
10  2023-01-11  -1.906933
11  2023-01-12  -3.536923
12  2023-01-13  -3.601052
13  2023-01-14  -1.967040
14  2023-01-15  -5.237476
15  2023-01-16  -4.902184
16  2023-01-17  -5.521366
17  2023-01-18  -2.939659
18  2023-01-19   2.074929
19  2023-01-20  -0.843776
20  2023-01-21  -6.764806
21  2023-01-22  -9.289010
22  2023-01-23 -13.184186
23  2023-01-24 -12.690851
24  2023-01-25  -7.230189
25  2023-01-26  -7.831264
26  2023-01-27 -10.850896
27  2023-01-28 -11.389762
28  2023-01-29  -8.032914
29  2023-01-30  -0.510242


In [16]:
submission.to_csv("DEAGYUL_16_submission.csv", index=False)

print(submission.head())

           일시      평균기온
0  2023-01-01 -1.695697
1  2023-01-02 -2.177952
2  2023-01-03  3.460828
3  2023-01-04  6.425697
4  2023-01-05 -3.931063
